## CS690D: Homework 1 (due 2/19/19 on Gradescope)




This homework contains both a written component and an implementation component. First, you'll answer a couple of questions about word embeddings (problems 4 and 5 from Eisenstein ch. 14).  Please format any equations or math in your answers using LaTeX; see the equations in the problem statements for examples. 

### 1.1 (10 pts): 
A simple way to compute a distributed phrase representation is to add up the distributed representations of the words in the phrase. Consider a sentiment analysis
model in which the predicted sentiment is, $\psi(w) = \theta · (\sum_{m=1}^M x_m)$, where $x_m$ is
the vector representation of word $m$. Prove that in such a model, the following two
inequalities cannot both hold:

<center>$\psi(\text{good}) >\psi(\text{not good})$</center>
<center>$\psi(\text{bad}) <\psi(\text{not bad})$</center>

#### write your answer here!!!

### 1.2(10 pts)
Now let’s consider a slight modification to the prediction model in the previous
problem:

<center>$\psi(w) = \theta \cdot \text{ReLu}( \sum_{m=1}^M x_m)$</center>

Show that in this case, it is possible to achieve the inequalities above. Your solution
should provide the weights $\theta$ and the embeddings $x_\text{good}, x_\text{bad},$ and $x_\text{not}$.

#### write your answer here!!!

### PART 2: IMPLEMENTING BACKPROP

In the remaining three problems, you will implement *backpropagation*  to train several different neural architectures. Specifically, your job is to compute the partial derivatives of the provided loss function with respect to each parameter of the network. Your work will be checked automatically against the output of pytorch's autograd. We will provide partial credit for each problem in the event that you are able to correctly compute some partial derivatives but not others.

**WARNINGS: DO NOT USE OUTPUT FROM AUTOGRAD IN YOUR COMPUTATIONS. DO NOT MODIFY ANY CODE OUTSIDE OF THE SPECIFIED BLOCKS WHERE YOU ARE TO IMPLEMENT BACKPROP. IF OUR GRADING SCRIPTS DETECT ANY VIOLATIONS, YOU WILL RECEIVE ZERO POINTS FOR THAT PROBLEM.**

Run the below cell to import pytorch and set up the gradient checking functionality.


In [ ]:
import torch
import torch.nn as nn
device = torch.device('cpu')

# checks equality between your gradients and those from autograd
# 'params' and 'your_gradient' are both dictionaries with the same keys, the names of parameters.
# 'params' contains model parameters augmented with pytorch's automatically-computed gradients.  
# 'your_gradient' contains tensors you calculated yourself.  This function assumes pytorch computed
# them correctly, and checks whether your calculations match.
def gradient_check(params, your_gradient):
    all_good = True
    for key in params.keys():
        if params[key].grad.size() != your_gradient[key].size():
            print('GRADIENT ERROR for parameter %s, SIZE ERROR\nyour size: %s\nactual size: %s\n'\
                % (key, your_gradient[key].size(), 
                   params[key].grad.size()))
            all_good = False
        elif not torch.allclose(params[key].grad, your_gradient[key], atol=1e-6):
            print('GRADIENT ERROR for parameter %s, VALUE ERROR\nyours: %s\nactual: %s\n'\
                % (key, your_gradient[key].detach(), 
                   params[key].grad))
            all_good = False
            
    return all_good


### 2.1: Warmup with single neurons (20 points)
The below cell trains a network with just single neurons in each layer on a small dataset of ten examples. We saw this same network architecture in class, so all you have to do is translate the partial derivatives we computed into code. As a reminder, the network is defined as:

<center>$\text{h} = \tanh(w_1 * \text{input})$</center>

<center>$\text{pred} = \tanh(w_2 * \text{h})$</center>

If you run the cell below, you should see "GRADIENT ERRORS". Once you implement the partial derivatives $\frac{\partial{L}}{\partial{w_1}}$ and $\frac{\partial{L}}{\partial{w_2}}$ correctly, you will instead see a "SUCCESS" message. 


In [ ]:
# initialize model parameters
params = {}
params['w1'] = torch.randn(1, 1, requires_grad=True) # input > hidden with scalar weight w1
params['w2'] = torch.randn(1, 1, requires_grad=True) # hidden > output with scalar weight w2

# set up some training data
inputs = torch.randn(20, 1)
targets = inputs / 2

# training loop
all_good = True
for i in range(len(inputs)):
    
    ## forward prop, then compute loss.
    a = params['w1'] * inputs[i] # intermediate variable, following lecture notes
    hidden = torch.tanh(a)
    b = params['w2'] * hidden
    pred = torch.tanh(b)
    loss = 0.5 * (targets[i] - pred) ** 2 # compute square loss
    loss.backward() # runs autograd
    
    ####################
    # TODO: IMPLEMENT BACKPROP HERE
    # DO NOT MODIFY ANY CODE OUTSIDE OF THIS BLOCK!!!!
    your_gradient = {}
    your_gradient['w1'] = torch.zeros(params['w1'].size()) # implement dL/dw1
    your_gradient['w2'] = torch.zeros(params['w2'].size()) # implement dL/dw2
    # END 
    ####################

    if not gradient_check(params, your_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['w1'].grad.zero_()
    params['w2'].grad.zero_() 
    
if all_good:
    print('SUCCESS! you passed the gradient check.')

### 2.2: Deep averaging network (30 points)
Now we're going to move to a more complex network featuring multiple matrix/vector operations. Instead of taking single numbers as input, this network takes in *a single sequence of word embeddings* associated with a sentence. We'll call the input $X$; it has dimensionality $N \times D$, where $N$ is the number of words in the sentence and $D$ is the word embedding dimensionality. We'll denote the $i^{th}$ word embedding in the sentence, which corresponds to the $i^{th}$ row of $X$, as $X_i$. The network is trained using softmax / cross entropy loss for sentiment analysis, so each input is associated with a target value $t$ (positive, negative, or neutral). The network is described by the following set of equations:

<center>$\text{ave} = \frac{1}{N} \sum_{i=0}^{N} X_i$</center>

<center>$\text{h} = \text{ReLu}(W_1 \cdot \text{ave})$</center>

<center>$\text{pred} = \text{softmax}(W_2 \cdot h)$</center>

where $\text{ReLu}(x) = max(0, x)$. We have provided you with the $\text{softmax}$ derivative, which you will use to compute three partial derivatives: $\frac{\partial{L}}{\partial{W_1}}$, $\frac{\partial{L}}{\partial{W_2}}$, and $\frac{\partial{L}}{\partial{X}}$. 

Just as in the previous problem when you run the cell below, you should see "GRADIENT ERRORS", which will be replaced with a "SUCCESS" message once the gradient has been correctly implemented.

In [ ]:
# let's set some hyperparameters first
N = 4 # all sentences will be of length 4
D = 5 # word embedding dim = 5
M = 3 # hidden dimensionality
labels = {'negative':0, 'neutral':1, 'positive':2}
vocab = {'really':0, 'movie':1, 'was':2, 'good':3, 'not':4, 'okay':5}
num_labels = len(labels)
len_vocab = len(vocab)

# initialize model parameters
params = {}
params['X'] = torch.randn(len_vocab, D, requires_grad=True)
params['W1'] = torch.randn(M, D, requires_grad=True) 
params['W2'] = torch.randn(num_labels, M, requires_grad=True) 

# set up some training data
inputs = [('positive', 'movie was really good'),
          ('neutral', 'really really really okay'),
          ('negative', 'movie was not good')]

# training loop
all_good = True
for i in range(len(inputs)):
    
    # obtain word embeddings for input sentence
    target, sentence = inputs[i]
    target = labels[target]
    input = torch.LongTensor(N)
    for j, w in enumerate(sentence.split()):
        input[j] = vocab[w]
    
    ## forward prop, then compute loss.
    ave = torch.zeros(D) # first let's compute the word embedding average
    for j in range(N):
        w_idx = input[j]
        ave += params['X'][w_idx]
    ave = ave / N
        
    # now we'll pass it through the DAN
    a = torch.mv(params['W1'], ave)
    h = torch.relu(a)
    b = torch.mv(params['W2'], h)
    pred = torch.softmax(b, 0)
    
    loss = -1 * torch.log(pred[target]) # negative log likelihood of target class
    loss.backward() # runs autograd
    
    # we give you the derivative for the softmax / CE loss as "dLdb"
    dLdb = pred.clone().detach()
    dLdb[target] -= 1.
    
    ####################
    # TODO: IMPLEMENT BACKPROP HERE
    # DO NOT MODIFY ANY CODE OUTSIDE OF THIS BLOCK!!!!
    your_gradient = {}
    your_gradient['W2'] = torch.zeros(params['W2'].size()) # implement dL/dW2
    your_gradient['W1'] = torch.zeros(params['W1'].size()) # implement dL/dW1
    your_gradient['X'] = torch.zeros(params['X'].size()) # implement dL/dX
    # END
    ####################
    
    if not gradient_check(params, your_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['X'].grad.zero_()
    params['W1'].grad.zero_()
    params['W2'].grad.zero_() 
    
if all_good:
    print('SUCCESS! you passed the gradient check.')

### 2.3: Recurrent neural network (30 points)
For the last (and most difficult) problem of this homework, we'll implement *backpropagation through time*,  an extension of backpropagation for recurrent neural networks. We'll stick with the same sentiment analysis example as before but change the network architecture from a DAN to a variant of an RNN with a multiplicative term. Remember that computations in an RNN proceed sequentially, so for inputs where $N=4$, we compute four different hidden states ($h_0, h_1, h_2, h_3$) and feed the final hidden state $h_3$ to the output layer. The network is then defined as:

<center>$h_i= \tanh(W_h \cdot h_{i-1} + W_x \cdot X_i + h_{i-1} * h_{i-2}$)</center>
<center>$\text{pred} = \text{softmax}(W_\text{out} \cdot h_3)$</center>

You'll need to correctly compute four partial derivatives to get the SUCCESS message: $\frac{\partial{L}}{\partial{W_h}}$, $\frac{\partial{L}}{\partial{W_x}}$, $\frac{\partial{L}}{\partial{W_\text{out}}}$,and $\frac{\partial{L}}{\partial{X}}$. 

##### Hint: It might be helpful to think of an RNN as a feed-forward network by "unrolling" it over the time dimension. 

In [ ]:
# initialize model parameters
params = {}
params['X'] = torch.randn(len_vocab, D, requires_grad=True)
params['Wx'] = torch.randn(M, D, requires_grad=True) 
params['Wh'] = torch.randn(M, M, requires_grad=True)
params['Wout'] = torch.randn(num_labels, M, requires_grad=True) 

# training loop
all_good = True
for i in range(len(inputs)):
    
    # obtain word embeddings for input sentence
    target, sentence = inputs[i]
    target = labels[target]
    input = torch.LongTensor(N)
    for j, w in enumerate(sentence.split()):
        input[j] = vocab[w]
    
    ## forward prop, then compute loss.
    hiddens = {} # stores hidden state / intermediate vars at each timestep
    for j in range(N):
        w_idx = input[j]
        hiddens[j] = {}
        
        # no previous hidden state, just project word embedding
        if j == 0:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx]) 
            
        elif j == 1:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx]) + \
                torch.mv(params['Wh'], hiddens[j-1]['h'])
            
        else:
            hiddens[j]['a'] = torch.mv(params['Wx'], params['X'][w_idx]) + \
                torch.mv(params['Wh'], hiddens[j-1]['h']) + \
                hiddens[j-1]['h'] * hiddens[j-2]['h']
            
        hiddens[j]['h'] = torch.tanh(hiddens[j]['a'])

    b = torch.mv(params['Wout'], hiddens[N-1]['h'])
    pred = torch.softmax(b, 0)
    
    loss = -1 * torch.log(pred[target]) # negative log likelihood of target class
    loss.backward() # runs autograd
    
    # we give you the derivative for the softmax / CE loss as "dLdb"
    dLdb = pred.clone().detach()
    dLdb[target] -= 1.
    
    ####################
    # TODO: IMPLEMENT BACKPROP HERE
    # DO NOT MODIFY ANY CODE OUTSIDE OF THIS BLOCK!!!!
    your_gradient = {}
    your_gradient['Wout'] = torch.zeros(params['Wout'].size()) # implement dL/dWout
    your_gradient['X'] = torch.zeros(params['X'].size()) # implement dL/dX
    your_gradient['Wx'] = torch.zeros(params['Wx'].size()) # implement dL/dWx
    your_gradient['Wh'] = torch.zeros(params['Wh'].size()) # implement dL/dWh
    # END
    ####################
            
    if not gradient_check(params, your_gradient):
        all_good = False
        break
    
    # zero gradients after each training example
    params['X'].grad.zero_()
    params['Wx'].grad.zero_()
    params['Wh'].grad.zero_() 
    params['Wout'].grad.zero_() 

    
if all_good:
    print('SUCCESS! you passed the gradient check.')

In [ ]:
# $